In [2]:
from pyspark.sql import SparkSession,DataFrame
import os
import numpy as np
from pyspark.sql import functions as f
from pyspark.sql.types import *
import io
import time
from pyspark.sql import Row

In [3]:
worker_image="inseefrlab/jupyter-datascience:py3.9.7-spark3.1.2"

local=False
if local:
    spark = SparkSession \
    .builder.master("local[4]") \
    .appName("SparkReadLargeDataSet") \
    .getOrCreate()
else: 
    spark = SparkSession \
    .builder.master("k8s://https://kubernetes.default.svc:443") \
    .appName("SparkReadLargeDataSet") \
    .config("spark.kubernetes.container.image", worker_image) \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
    .config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME']) \
    .config("spark.executor.instances", "8") \
    .config("spark.executor.memory","8g") \
    .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
    .getOrCreate()

2021-10-25 15:41:24,222 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-10-25 15:41:25,260 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
spark.sparkContext.stop()

In [1]:
! kubectl get pods

I1025 15:40:59.898017    7750 request.go:665] Waited for 1.173933199s due to client-side throttling, not priority and fairness, request: GET:https://kubernetes.default/apis/discovery.k8s.io/v1beta1?timeout=32s
NAME                                               READY   STATUS      RESTARTS   AGE
amundsen-elasticsearch-master-0                    1/1     Running     0          5d5h
amundsen-elasticsearch-master-1                    1/1     Running     0          5d5h
amundsen-elasticsearch-master-2                    1/1     Running     0          5d5h
amundsen-test-frontend-5648d65454-ffgpt            1/1     Running     0          5d5h
amundsen-test-metadata-c57c47666-7bjsz             1/1     Running     0          5d5h
amundsen-test-search-5c8455d784-g56vh              1/1     Running     0          5d5h
datahub-datahub-frontend-6ff5494f44-7gcpx          1/1     Running     0          5d
datahub-datahub-gms-5b66978446-g2526               1/1     Running     0          5d
datahub-data

In [7]:
data_path="s3a://pengfei/diffusion/data_format/ny_taxis/parquet/raw_2011_2012"

In [ ]:
df=spark.read.parquet(data_path)
df.show()